In [ ]:
def lasso_fs():
    result_df = pd.read_csv('Advanced_AF_DER_Dataset.csv', index_col=None)
    print(len(result_df))
    result_df['date'] = pd.to_datetime(result_df['date'])
    result_df['Year'] = result_df['date'].dt.year
    result_df['Month'] = result_df['date'].dt.month
    result_df['DayOfWeek'] = result_df['date'].dt.dayofweek
    X = result_df.drop(['Ticker', 'date', 'Implosion_Next_Year'], axis=1)
    cols = X.columns
    print(X.head())
    Y = result_df['Implosion_Next_Year']

    tscv = TimeSeriesSplit(n_splits=5)

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        alpha = 0.01
        lasso_model = Lasso(alpha=alpha)
        lasso_model.fit(X_train_scaled, y_train)

        feature_importances = lasso_model.coef_

        feature_importance_df = pd.DataFrame({'Feature': cols, 'Importance': feature_importances})

        feature_importance_df['Absolute Importance'] = feature_importance_df['Importance'].abs()
        feature_importance_df = feature_importance_df.sort_values(by='Absolute Importance', ascending=False).drop('Absolute Importance', axis=1)
        print(f'Feature Importances for Fold {tscv.get_n_splits()}:')
        print(feature_importance_df)
        
        
def boruta_fs():
    result_df = pd.read_csv('Advanced_AF_DER_Dataset.csv', index_col=None)
    X = result_df.drop(['Ticker', 'date', 'Implosion_Next_Year'], axis=1)
    Y = result_df['Implosion_Next_Year']
    scaler = StandardScaler()
    scaler.fit(X)
    X=scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    
    model = xgb.XGBClassifier()
    feat_selector = BorutaPy(model, n_estimators='auto', verbose=2, random_state=1)
    feat_selector.fit(X_train, y_train)
    
    print(feat_selector.support_)
    print(feat_selector.ranking_)
    
    
